# Lab 1 -- RSA basics and getting started
This lab will take you through the basics of encryption and decryption of RSA

There will be quite a few things we will need to talk about, so get prepared and get excited!

This, and subsequent labs, will try to work from simpler concepts to more complex.  One problem we find with working with RSA is you need to have a strong understanding of multiple concepts, some of the concepts will be foreign to you unless you were a double Math / CompSci major.

This is good news / bad news, you may not be familiar with all the concepts, but you should be familiar with some.  If you feel unfamiliar with none, then with a small amount of catch up you should be right on par.

This lab will cover some ancillary ideas which will lead to everything coming together in the end.

Of note, this lab is intended to cover the implementation of RSA, so you can see it work in practice.  Our hope is that we will start with small easy to handle numbers, and you can see that the concepts aren't inherently difficult.  We will add increasingly larger numbers and increasingly professional implementations, and you should see that nothing really changed except for the sizes of numbers.  But since we are using a computer the size of the number is immaterial to us.  
{We use SageMath since the size of the number is hugely material to the computer! And we can easily overrun memory if we aren't careful}

***NOTE***  
*The material and functions contained within will be required for the midterm.*

### RSA ToDo List
Alice:  
1. Declaring a _String_ to be encrypted  
2. _Encoding_ the string so it can be encrypted  
3. _Encrypting_ the encoding to get ciphertext  

Bob:  
1. _Decrypting_ ciphertext to get encoding  
2. _Decoding_ encoding to get plaintext  

### How does RSA Work?
RSA uses some "simple" mathematical operations for encryption and decryption.
The first operation is exponentiation.
The second operation is reduction by modulus.

When we say exponentiation we are quite simply taking a number, $m$, and raising it to a power, $e$, so we get $m^e$

The nomenclature:  
$m$ is the message to be encrypted.  (In this class this will need to be the encoded plaintext, but more on that later.)  
$e$ is an encryption exponent.  This is also called the public key, since $e$ will be made publicly available.  
$d$ is a decryption exponent.  This is also called the private key, since no one except the keyholder should be able to decrypt a transmisison.  

In [7]:
################
# DATA DATA DATA
################


# for now we will use these values for RSA.
# I am including quite a few values here, in case you want to explore
# the only ones we will use now are Ea, Da, and Na

# Alice data
Pa = 2**107 - 1
Qa = 2**607 - 1
Na = Pa*Qa                    # Alice modulus
phi_Na = (Pa - 1)*(Qa - 1)
Ea = 101                      # Alice encryption exponent
Da = inverse_mod(Ea, phi_Na)  # Alice decryption exponent

# Bob data
Pb = 2**521 - 1
Qb = 2**127 - 1 
Nb = Pb*Qb                    # Bob modulus
phi_Nb = (Pb - 1)*(Qb - 1)
Eb = 173                      # Bob encryption exponent
Db = inverse_mod(Eb, phi_Nb)  # Bob decryption exponent

# a standard RSA key will have two parts, a public and private part
# here the public key will be the encryption exponent, and the modulus
# <Ea, Na> is the public key of Alice
# the private key Alice doesn't tell anyone, is <Da, Na>

### RSA ToDo List
Alice:  
1. **Declaring a _String_ to be encrypted**
2. _Encoding_ the string so it can be encrypted  
3. _Encrypting_ the encoding to get ciphertext  

Bob:  
1. _Decrypting_ ciphertext to get encoding  
2. _Decoding_ encoding to get plaintext  

Since we need to perform math to encrypt and decrypt in RSA, we need to use numbers.  
Say I want to transmit a secret message like, `"Have a nice day."`  
Well RSA says we simply raise this message to an encryption exponent.  Does it make sense to do this then: 

$\big("Have\ a\ nice\ day."\big)^e$  

Hopefully not.  We need to convert this string `"Have a nice day."` into an integer value of some sort so using an exponent makes sense..  This converstion is called encoding.  
Encoding occurs all the time in computers, ASCII is a common encoding, although for our purposes straight ASCII can be problematic.  
In fact, there are little problems which arise all the time when trying to find a good encoding scheme.  
We will be playing with `openssl` later, and it uses a common encoding called `-base64`.

For now we will make it easy for you.  I have devised a proprietary encoding called W202.  You can use that for now by calling the function `W202_encode` and `W202_decode`.  These functions translate between strings and integers.  There is a catch though.  ***You cannot use lowercase letters with these functions.*** I could build that functionality in, but it's going to overcomplicate what is supposed to be a simplified scenario.

### RSA ToDo List
Alice:  
1. Declaring a _String_ to be encrypted
2. **_Encoding_ the string so it can be encrypted**  
3. _Encrypting_ the encoding to get ciphertext  

Bob:  
1. _Decrypting_ ciphertext to get encoding  
2. _Decoding_ encoding to get plaintext 

In [8]:
####################################################
# The following functions are written and functional 
# They are for your use
####################################################


# function to take ascii plaintext and convert to decimal value message
# no lowercase letters allowed
def encode_message(message):
    encoded_message = 0

    for char in message:
        encoded_message = 100*encoded_message + ord(char)

    return encoded_message

# function to take decimal value message and convert to ascii plaintext
# should return only uppercase letters
def decode_message(message):
    decoded_message = ""

    while message > 0:
        decoded_message = chr(message%100) + decoded_message
        message = message//100

    return decoded_message


In [9]:
################
# DEMO DEMO DEMO
################


# Let's check to see if the encoding works for us

# make a string
message = "HELLO WORLD"

# let's see what type of varialbe message is
print(type(message))

# You can see in the output that message is of the 
# class 'str' which means string.  Woot.

# now let's encode and turn it into an integer
encoded_message = encode_message(message)

# check type
print(type(encoded_message))

# print message and encoded_message together
print()
print(message)
print(encoded_message)

<class 'str'>
<class 'sage.rings.integer.Integer'>

HELLO WORLD
7269767679328779827668


Just to talk about the encoding a bit.  
The simplified code I used finds the two digit ascii value.  You can see this if you look up an ascii table, which gives this type of encoding.  

For instance, the letter `H` has an ascii value of 72. The `E` has a value of 69, `L` a value of 76, etc...  

In this way we can convert back and forth between this integer and a readable string.

*TECHNICAL NOTE*. 
It is important to check your encoding against your data types.  This is supposed to be a simplified example, but mistakes are still possible.  So, for our purposes you should always be using a _string_ as your message.  For our encoding it is just fine to use integers and numbers all you want.  So you can encode the string "1232221" just fine.  But you won't be able to encode an integer valued at 1232221. You can mix and match numbers in your string also, for instance you can say "HELLO, THE SECRET MESSAGE IS 1232221, AND I AM A SPY!". This is fine.  Enjoy!

In [15]:
################
# YOUR WORK HERE
################


# practice encoding some strings yourself
# create 3 messages below, and encode them.
# print all the outputs in a professional manner.

jason_one = "THE MATH BEHIND CRYPTOGRAPHY IS AWESOME"
jason_two = "STARTING OPEN MICS PODCAST SOON"
jason_three = "THE IAM SLACK CHANNEL IS GREAT"

encoded_jason_one = encode_message(jason_one)
encoded_jason_two = encode_message(jason_two)
encoded_jason_three = encode_message(jason_three)

print(jason_one)
print(encoded_jason_one)
print()
print(jason_two)
print(encoded_jason_two)
print()
print(jason_three)
print(encoded_jason_three)

# finally encode these three messages
message_one = 'I LOVE MACARONI AND CHEESE'
message_two = 'I LOVE SAMOSAS AND LASSI'
message_three = 'I LOVE KIM CHI JI GAE'

encoded_message_one = encode_message(message_one)
encoded_message_two = encode_message(message_two)
encoded_message_three = encode_message(message_three)

print()
print(message_one)
print(encoded_message_one)
print()
print(message_two)
print(encoded_message_two)
print()
print(message_three)
print(encoded_message_three)

# take those three messages and print them an their 
# corresponding encoding in a professional manner

THE MATH BEHIND CRYPTOGRAPHY IS AWESOME
847269327765847232666972737868326782898084797182658072893273833265876983797769

STARTING OPEN MICS PODCAST SOON
83846582847378713279806978327773678332807968676583843283797978

THE IAM SLACK CHANNEL IS GREAT
847269327365773283766567753267726578786976327383327182696584

I LOVE MACARONI AND CHEESE
7332767986693277656765827978733265786832677269698369

I LOVE SAMOSAS AND LASSI
733276798669328365777983658332657868327665838373

I LOVE KIM CHI JI GAE
733276798669327573773267727332747332716569


### RSA ToDo List
Alice:  
1. Declaring a _String_ to be encrypted
2. _Encoding_ the string so it can be encrypted  
3. **_Encrypting_ the encoding to get ciphertext**  

Bob:  
1. _Decrypting_ ciphertext to get encoding  
2. _Decoding_ encoding to get plaintext 

Let's do some encrypting.  After all of this setup, the actual encryption is quite straightforward.  

RSA is executed by taking the message and raising it to an exponent, and finally reducing by a modulus.  

If we let m be the message (encoded already so it is integer valued), then raising it to an exponent is $m^e$. 

Finally, reducing by a modulus can be accomplished, in general by the modulus operator %, so we can say `m^e%N`.  

There is one problem with this.  We are using sagemath kernel (or should be, check your kernel right now!) which allows us access to faster and better processes.  So instead of the slow and error prone `m^e%N` we will use the sagemath `power_mod()` function.  This takes a value, raises it to a power, and finally simplifies by a modulus... exactly what we want.

In [16]:
################
# DEMO DEMO DEMO
################


# Alice wants to send a message to Bob
# she's encocded it already as encoded_message above
# since she is sending a message to Bob, she will use Bob's 
# public encryption exponent, Eb, and his public modulus Nb

ciphertext = power_mod(encoded_message, Eb, Nb)

# what does the ciphertext look like?
# well we expect it to be a large integer
print("ciphertext: ")
print(ciphertext)

ciphertext: 
1037069157757956994625823706189999461002081547854960463033396282491525632562203613727527764497625554799688861472050897908386422690363305201977550903757763407511029533346932195064195005692467419473


### RSA ToDo List
Alice:  
1. Declaring a _String_ to be encrypted
2. _Encoding_ the string so it can be encrypted  
3. _Encrypting_ the encoding to get ciphertext  

Bob:  
1. **_Decrypting_ ciphertext to get encoding**  
2. _Decoding_ encoding to get plaintext 

Now Alice has sent an encrypted message to Bob.  This message can be intercepted and no one will be able to read it unless they have access to Bob's private key, Db, and Bob's modulus Nb.

So let's decrypt it!

RSA decryption is as simple as taking the ciphertext and raising it to the decryption exponent and simplifying by the modulus.

We use power_mod() again.  Super easy.  Just remember that this will return the encoded message, so that will be the final step.

In [17]:
# decrypt ciphertext received from Alice
decrypted_message = power_mod(ciphertext, Db, Nb)

# what does the decrypted (but not decoded) message look like?
print("decrypted_message:")
print(decrypted_message)

# does this look like the encoded_message Alice encrypted?

decrypted_message:
7269767679328779827668


### RSA ToDo List
Alice:  
1. Declaring a _String_ to be encrypted
2. _Encoding_ the string so it can be encrypted  
3. _Encrypting_ the encoding to get ciphertext  

Bob:  
1. _Decrypting_ ciphertext to get encoding  
2. **_Decoding_ encoding to get plaintext** 

In [18]:
# let's decode this message and find out what Alice has to say!
plaintext_message = decode_message(decrypted_message)
print(plaintext_message)

HELLO WORLD


Great, now it's time to try yourself!

In [19]:
# 1 -- create a message you want to send to Bob
# call it my_message
my_message = "JASON CHOW SPRING 2022"

# print my_message
print(my_message)

JASON CHOW SPRING 2022


In [20]:
# 2 -- encode the string, save the encoding as my_encoded
my_encoded = encode_message(my_message)

# print my_encoded
print(my_encoded)

74658379783267727987328380827378713250485050


In [21]:
# 3 -- encrypt the encoding, save it as my_ciphertext
my_ciphertext = power_mod(my_encoded, Eb, Nb)

# print my_ciphertext
print(my_ciphertext)

864050619642996868064972675637370284090193239839759284072845909300152742818343576595221382009302822603865714842694175736240459984360506456031476879308948324721763317747466473095236758334673475669


In [22]:
# 4 -- now act like bob and verify that he can read your message
# decrypt the message and save it as bob_encoded
bob_encoded = power_mod(my_ciphertext, Db, Nb)

# print bob_encoded
print(bob_encoded)

74658379783267727987328380827378713250485050


In [23]:
# 5 -- finally act like bob and decode the message
jason_message = decode_message(bob_encoded)

# print decoded message
print(jason_message)

JASON CHOW SPRING 2022


# Digital Signatures
Let us now use these RSA principles to implement RSA digital signatures.  
At this point digital signatures should be pretty straightforward. But there are a few things to pay attention to.

First off, to create a digital signature you will need to take the hash of your message.  
Finally you will encrypt that message with your private key.  

**note**. 
Hashes here can be tricky because you need to focus on the datatype.  A hash will commonly be viewed as a hexadecimal value, meaning a number which uses 0-9 as well as letters a-f.  If we are comparing hashes, we usually want to print the values out, and printing a hash is done as a hexadecimal number, or sometimes it's convenient to print as a string.  Just be careful and pay attention to the details.  
Like before, if we are going to encrypt or decrypt, or use `power_mod()` the value needs to be a decimal integer.

In [24]:
##################################################
# The following function is written and functional 
# It is for your use
##################################################


# here is a function called hash_message() which takes as an argument a
# string message and returns a hexadecimal hash

def hash_message(message):
    import hashlib                        # gives us access to sha256()
    hash = hashlib.sha256()               # instantiates a sha256 object
    hash.update(message.encode('UTF-8'))  # creates hash of the message
    return hash.hexdigest()               # returns hexadecimal value

In [25]:
# create a message
message = "THIS IS AN ENCRYPTED MESSAGE, BUT I ALSO WANT TO HAVE A DIGITAL SIGNATURE"
hash = hash_message(message)
print("This is what a hash of the message looks like: \n\t" + str(hash))
print()

# but to create a digital signature we need to apply a private exponent
# so we change this hexadecimal value into a decimal value 
# use int() function with hash, and tell it that you are using base 16
hash_int = int(hash, 16)
print("hash_int: \n\t" + str(hash_int))
print()

# you can see that you have a long integer now!
# so Alice's the digital signature of this message is
digital_signature = power_mod(hash_int, Da, Na)
print("digital signature: \n\t" + str(digital_signature))

This is what a hash of the message looks like: 
	9eee1efc448603fbe24842e54dbe49a6cae3e45165ae1785243261a1743b93d8

hash_int: 
	71886153531086675310348136005252694423735655382735726996747841801751726101464

digital signature: 
	46518795071408048579349189745128340929985898733161326139643286942902994811340532978293103660495936442202571297708543338271348674374492892927917995460537465463100053814447992925617416935116658700694542463649554783735


### How can you verify a digital signature?
Well if Bob receives this digital signature from Alice, he will also receive the message from her.  
So Bob can use sha256 to hash the message he receives.  
Finally he can compare that hash to the one he receives from Alice, once he'd decrypted her hash.

In [26]:
################
# DEMO DEMO DEMO
################


# Alice has sent Bob message from the previous codeblock and digital_signature
# Since Bob has received message he can quickly calculate the hash for himself
bob_hashed = hash_message(message)

# Bob can decrypt the digital signature from alice quickly using her public key
decrypted_signature = power_mod(digital_signature, Ea, Na)

# Bob needs to convert back to a hexadecimal value from an integer too
# this allows the hash to look like a hash, since they are hex values
hash = hex(decrypted_signature)[2:]     # the [2:] is a formatting gimmick

# finally Bob wants to validate that the hash he computed, bob_hashed
# matched the hash Alice sent
print(bob_hashed)
print(hash)

print("It is__ " + str((bob_hashed == hash)) + " __the signature is valid")

9eee1efc448603fbe24842e54dbe49a6cae3e45165ae1785243261a1743b93d8
9eee1efc448603fbe24842e54dbe49a6cae3e45165ae1785243261a1743b93d8
It is__ True __the signature is valid


In [27]:
################
# DATA DATA DATA
################


####################################
# Use the following RSA information:
####################################

# Alice data
Pa = 2**107 - 1
Qa = 2**607 - 1
Na = Pa*Qa                    # Alice modulus
phi_Na = (Pa - 1)*(Qa - 1)
Ea = 101                      # Alice encryption exponent
Da = inverse_mod(Ea, phi_Na)  # Alice decryption exponent

# Bob data
Pb = 2**521 - 1
Qb = 2**127 - 1 
Nb = Pb*Qb                    # Bob modulus
phi_Nb = (Pb - 1)*(Qb - 1)
Eb = 173                      # Bob encryption exponent
Db = inverse_mod(Eb, phi_Nb)  # Bob decryption exponent

Take the message `I SPY WITH MY LITTLE EYE...`  
If Alice encrypts this message for Bob, then what is the ciphertext?  
If Alice then signs the message for Bob, then what is the digital signature?  


In [54]:
#Alice's message encoded and encrypted with Bob's public key and modulus.
alice_message = "I SPY WITH MY LITTLE EYE..."
alice_message_encoded = encode_message(alice_message)
alice_ciphertext = power_mod(alice_message_encoded, Eb, Nb)
print("The ciphertext is: ")
print(alice_ciphertext)
print()

#Alice's digital signature.
alice_hashed = hash_message(alice_message)
alice_hash_int = int(alice_hashed, 16)
alice_digital_signature = power_mod(alice_hash_int, Da, Na)
print("Alice's digital signature is: ")
print(alice_digital_signature)

The ciphertext is: 
53742469989810864839181660204999891446353054564426860900441023292868223213456660042471628938539072827970949872990686245480983046461079727964706674977227138481903057034826921531390485925416961723

Alice's digital signature is: 
862950513856659056935532167585793266547447251155284967436358471152599094486060680627608115303119589152310657900870990695725745360684848237017506700575054805926766548601217385293511016443292725845886536398545483040


Take the message `MY SPY EYE SPIES LITTLE FLIES!`  
If Bob encrypts this message for Alice, what is the ciphertext?  
If Bob then signs the message for Alice, what is the digital signature  

In [51]:
#Bob's message encoded and encrypted with Alice's public key and modulus.
bob_message = "MY SPY EYE SPIES LITTLE FLIES!"
bob_message_encoded = encode_message(bob_message)
bob_ciphertext = power_mod(bob_message_encoded, Ea, Na)
print("The ciphertext is: ")
print(bob_ciphertext)
print()

#Bob's digital signature.
bob_hashed = hash_message(bob_message)
bob_hash_int = int(bob_hashed, 16)
bob_digital_signature = power_mod(bob_hash_int, Db, Nb)
print("Bob's digital signature is: ")
print(bob_digital_signature)

The ciphertext is: 
70605676389401761195232655159321554409289827427935280863828631557646279530147951672632630563441231705133487794101193940636829779794821324378149918142485466425174391627121380644768290803372381627775525371562907993313

Bob's digital signature is: 
801645054437377253503116839794451716866122206541689623408048859211978714539324737877521640425587421546150352885614516424444611827656842950498919288976780210109807283728104369438837955388274854435


Finally validate a signature.  
Say you are Alice, and you receive this digitally signed message:  
message = `WE NEED TO OBTAIN THE DATA FILE ONE IMMEDIATELY`  
signature = `84635300824201233164756878515597679151878722454899243802833883416142558505740809517766086977410665779047598931877880476387627089954879563464433736421917953336255847983780126364282104831007008610`
(note: signature is a long value, and likely is running off the  
screen, ensure you capture it all or properly use the variable)  
Is this signature valid? Show your work.

In [84]:
#Message and signature received by Alice.
message = "WE NEED TO OBTAIN THE DATA FILE ONE IMMEDIATELY"
signature = 84635300824201233164756878515597679151878722454899243802833883416142558505740809517766086977410665779047598931877880476387627089954879563464433736421917953336255847983780126364282104831007008610

#Alice hashing message, and validating signature using Bob's public key. 
alice_hashes_message = hash_message(message)
decrypted_signature = power_mod(signature, Eb, Nb)
hash = hex(decrypted_signature)[2:]
print("Is the signature value (True/False): ", str(alice_hashes_message == hash))
print()

#Printing Alice's hash and the hash provided for visual comparison.
#print("Alice's hash: ", alice_hashes_message)
#print("The hash provided: ", hash)



Is the signature value (True/False):  False

